## **Modelo Árbol de decisión mejorado**

Importar librerías necesarias

In [3]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

from scipy.sparse import hstack

In [4]:
# Guardando el dataset como un dataframe
df = pd.read_csv("/content/df16to25.csv")
df.head()

,id,stars,review_body,review_title,language,product_category,lenght_review_body,lenght_review_title,lenght_product_category
0,151766,4,Mando aceptable precio un poco caro.. pero cum...,Cumple su funcion,es,electronics,134,17,11
1,10209,5,Muy chula y se ha pegado muy bien al maletero ...,Bruja grande,es,automotive,75,12,10
2,197887,5,Me gustó mucho como idea para regalos funcionó...,Idea genial,es,home,51,11,4
3,68193,1,"A los seis meses de uso deja de funcionar, es ...",Decepcionante,es,wireless,64,13,8
4,149127,3,"Para un rato vale,pero no agarra mucho y para ...",Que para algunos juegos no vale,es,video_games,97,31,11


Crear la variable Sentimiento

In [5]:
def sentimiento(stars):
    if stars <= 2:
        return 'Negativo'
    elif stars == 3:
        return 'Neutro'
    else:
        return 'Positivo'

df['sentiment'] = df['stars'].apply(sentimiento)

df.head()

,id,stars,review_body,review_title,language,product_category,lenght_review_body,lenght_review_title,lenght_product_category,sentiment
0,151766,4,Mando aceptable precio un poco caro.. pero cum...,Cumple su funcion,es,electronics,134,17,11,Positivo
1,10209,5,Muy chula y se ha pegado muy bien al maletero ...,Bruja grande,es,automotive,75,12,10,Positivo
2,197887,5,Me gustó mucho como idea para regalos funcionó...,Idea genial,es,home,51,11,4,Positivo
3,68193,1,"A los seis meses de uso deja de funcionar, es ...",Decepcionante,es,wireless,64,13,8,Negativo
4,149127,3,"Para un rato vale,pero no agarra mucho y para ...",Que para algunos juegos no vale,es,video_games,97,31,11,Neutro


Crear Dataframe separados por estrellas

In [6]:
df_1 = df[df['stars'] == 1]
df_2 = df[df['stars'] == 2]
df_3 = df[df['stars'] == 3]
df_4 = df[df['stars'] == 4]
df_5 = df[df['stars'] == 5]

Seleccionar variables más relevantes para el modelo

In [7]:
X_texto = df['review_body']          # Texto del comentario
X_longitud = df[['lenght_review_body']]  # Longitud del comentario
y = df['sentiment']                  # Variable objetivo

Vectorización del texto

In [8]:
vectorizer_tree = TfidfVectorizer(
    max_features=300,   # limita el número de palabras
    ngram_range=(1,1),  # solo palabras individuales
    min_df=2            # ignora palabras que aparecen solo una vez
)

X_texto_vec = vectorizer_tree.fit_transform(X_texto)

Unir o concatenar las columnas texto(review body) + longitud del comentario

In [9]:
X_final = hstack([X_texto_vec, X_longitud.values])

Separando los datos de entrenamiento y prueba para el modelo

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X_final,
    y,
    test_size=0.2,
    stratify=y,      # mantiene proporción de clases de las distintas categorias de comentarios (Positivo, negativo, neutro)
    random_state=42
)

Creando el modelo del Árbol de Decisión (controlado o mejorado)

In [11]:
tree_model = DecisionTreeClassifier(
    max_depth=5,          # limita la profundidad del árbol para evitar que no pueda generalizár con nuevos datos
    min_samples_leaf=5,   # evita hojas con muy pocos datos
    class_weight='balanced',
    random_state=42
)

Entrenando el modelo Árbol de Decisión (mejorado)

In [12]:
#El Árbol va a aprender reglas a partir de los datos o registros sumnistrados en el dataframe
tree_model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5, min_samples_leaf=5,
                       random_state=42)

Se procede a evaluar el Modelo

In [13]:
# Para resaltar la idea es que el modelo prediga las 3 clases, tambien que no todo sea neutro y que el accuracy sea honesto (0.45–0.55)
y_pred_tree = tree_model.predict(X_test)

accuracy_tree = accuracy_score(y_test, y_pred_tree)

print("Accuracy Árbol de Decisión:", round(accuracy_tree, 2))
print(classification_report(y_test, y_pred_tree))

Accuracy Árbol de Decisión: 0.5
              precision    recall  f1-score   support

    Negativo       0.33      0.67      0.44         6
      Neutro       0.67      0.40      0.50         5
    Positivo       0.80      0.44      0.57         9

    accuracy                           0.50        20
   macro avg       0.60      0.50      0.51        20
weighted avg       0.63      0.50      0.52        20



Se crea la función limpiar texto

In [14]:
# Se importa la biblioteca re para normalizar el texto y evitar que el modelo trate como diferentes palabras que en realidad son iguales.
import re

def limpiar_texto(texto):
    """
    Limpia el texto:
    - Convierte a minúsculas
    - Elimina caracteres especiales
    - Elimina espacios extras
    """
    texto = texto.lower()
    texto = re.sub(r'[^a-záéíóúñ\s]', '', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

Se crea la función para la predicción del sentimiento

In [15]:
def predecir_sentimiento_arbol(texto):
    # Limpieza del texto para normalizarlo
    texto_limpio = limpiar_texto(texto)

    # Vectorización del texto
    texto_vec = vectorizer_tree.transform([texto_limpio])

    # Esta es la variable numérica: longitud del texto
    longitud = np.array([[len(texto_limpio)]])

    # Unión o contatenación de features o caracterés
    X_nuevo = hstack([texto_vec, longitud])

    # Predicción
    sentimiento = tree_model.predict(X_nuevo)[0]

    return f"Sentimiento (Árbol de Decisión): {sentimiento} | Precisión del modelo: {round(accuracy_tree, 2)}"

**Prueba del modelo**

In [20]:
#El modelo del Árbol de decisión (Controlado o mejorado) se mantiene con una precisión de 0.55 pero al igual que su primera versión aún continua sin reconocer adecuadamente los sentimientos negativos
predecir_sentimiento_arbol("Lamentable")


'Sentimiento (Árbol de Decisión): Negativo | Precisión del modelo: 0.5'

**Métricas** Precisión(Accuracy), Recall(Sensibilidad), F1-Score(Puntuación única(de 0 a 1) que indica el rendimiento general de un modelo de clasificación).

In [17]:
print(classification_report(y_test, y_pred_tree))

              precision    recall  f1-score   support

    Negativo       0.33      0.67      0.44         6
      Neutro       0.67      0.40      0.50         5
    Positivo       0.80      0.44      0.57         9

    accuracy                           0.50        20
   macro avg       0.60      0.50      0.51        20
weighted avg       0.63      0.50      0.52        20

